In [1]:
import pandas as pd
import numpy as np
import re
import plotly.express as px

In [2]:
# Load the main CSV
df = pd.read_csv("RozeePK-Jobs-2024.csv")
df.head()

,Title,Salary,Job Type,Job Location,Functional Area,Career Level,Apply Before,Minimum Experience,Minimum Education,Gender,Age,Skills
0,Order Fulfillment Manager,"PKR. 30,000 - 60,000/Month",Full Time/Permanent,"Lahore,\n ...",Retail,Experienced Professional,3-Jan-25,2 Years,Intermediate/A-Level,Male,NaN,"Team Building, Order Tracking, Order Processin..."
1,Enterprise Business Development Manager,NaN,Full Time/Permanent,"Lahore,\n ...",NaN,NaN,19-Jan-25,3 Years,NaN,No Preference,NaN,"Relationship Building, Sales Acumen, Market Re..."
2,Video Editor,"PKR. 30,000 - 30,000/Month",Full Time/Permanent,"Karachi,\n ...",NaN,NaN,19-Jan-25,1 Year,NaN,No Preference,NaN,Video Editor
3,Branch in Charge (Patokki),"PKR. 60,000 - 60,000/Month",Full Time/Permanent,"Patoki,\n ...",NaN,NaN,19-Jan-25,3 Years,NaN,No Preference,NaN,"Customer Service, Problem Solving, Communicati..."
4,Senior PHP Developer,NaN,Full Time/Permanent,"Lahore,\n ...",NaN,NaN,19-Jan-25,3 Years,NaN,No Preference,NaN,"PHP, MySQL, Javascript, Laravel, HTML, CSS, Gi..."


# Cleaning Data to Prepare for Dash Board

In [3]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

In [4]:
#Drop Irrelevant Columns
df = df.drop(columns=["apply_before", "age", "gender"])
df.head()

,title,salary,job_type,job_location,functional_area,career_level,minimum_experience,minimum_education,skills
0,Order Fulfillment Manager,"PKR. 30,000 - 60,000/Month",Full Time/Permanent,"Lahore,\n ...",Retail,Experienced Professional,2 Years,Intermediate/A-Level,"Team Building, Order Tracking, Order Processin..."
1,Enterprise Business Development Manager,NaN,Full Time/Permanent,"Lahore,\n ...",NaN,NaN,3 Years,NaN,"Relationship Building, Sales Acumen, Market Re..."
2,Video Editor,"PKR. 30,000 - 30,000/Month",Full Time/Permanent,"Karachi,\n ...",NaN,NaN,1 Year,NaN,Video Editor
3,Branch in Charge (Patokki),"PKR. 60,000 - 60,000/Month",Full Time/Permanent,"Patoki,\n ...",NaN,NaN,3 Years,NaN,"Customer Service, Problem Solving, Communicati..."
4,Senior PHP Developer,NaN,Full Time/Permanent,"Lahore,\n ...",NaN,NaN,3 Years,NaN,"PHP, MySQL, Javascript, Laravel, HTML, CSS, Gi..."


In [5]:
df.shape

(1059, 9)

## Removing Null values

In [6]:
df.isna().sum()

title                  25
salary                314
job_type               25
job_location           25
functional_area       163
career_level          163
minimum_experience    312
minimum_education     196
skills                 26
dtype: int64

In [7]:
#Removing Null values from "Title" & "Apply Before"
df = df.dropna(subset=["title", "job_location"])

In [8]:
# Null value in Functional Area ,Career Level  &  Skills 
df["functional_area"] = df["functional_area"].fillna("Other").str.title()
df["career_level"] = df["career_level"].fillna("Not Specified")
df["skills"] = df["skills"].fillna("Not Specified")

## Cleaning salary column

In [9]:
# Cleaning salary column
def salary_to_mean(s):
    if pd.isna(s):
        return np.nan

    # Extract numbers
    nums = re.findall(r"\d[\d,]*", s)
    nums = [int(x.replace(",", "")) for x in nums]

    if len(nums) == 2:
        return np.mean(nums)
    elif len(nums) == 1:
        return nums[0]
    else:
        return np.nan

df["Salary"] = df["salary"].apply(salary_to_mean)

# Replace NaN with overall mean
df["Salary"] = df["Salary"].fillna(df["Salary"].mean())
df = df.drop(columns=["salary"])

## Cleaning Job Type Column

In [10]:
# Cleaning Job Type Column
def clean_job_type(x):
    if pd.isna(x):
        return np.nan
    x = x.lower()
    if "full" in x:
        return "Full Time"
    if "part" in x:
        return "Part Time"
    return "Other"

df["Job Type"] = df["job_type"].apply(clean_job_type)

# Remove null job-type rows
df = df.dropna(subset=["Job Type"])
df = df.drop(columns=["job_type"])

## Cleaning Job Location Column

In [11]:
# cleaning Job Location Column
valid_cities = [
    "Karachi","Lahore","Islamabad","Rawalpindi","Faisalabad","Peshawar",
    "Multan","Quetta","Hyderabad","Gujranwala","Sialkot","Bahawalpur",
    "Sargodha","Sukkur","Abbottabad","Mardan","Swat","Okara","Kasur",
    "Sheikhupura","Nawabshah","Mirpur","Gujrat","Rahim Yar Khan","Attock"
]

pattern = r"\b(" + "|".join(valid_cities) + r")\b"

def extract_city(text):
    if pd.isna(text):
        return np.nan
    m = re.search(pattern, text, flags=re.IGNORECASE)
    if m:
        return m.group(1)
    return "Other"

df["City"] = df["job_location"].apply(extract_city)
df = df.dropna(subset=["City"])
df = df.drop(columns=["job_location"])

## Cleaning Experience Required Column

In [12]:
# cleaning Experience Required Column
df = df.copy()
def exp_to_num(x):
    if pd.isna(x):
        return np.nan
    m = re.search(r"(\d+)", str(x))
    return int(m.group(1)) if m else np.nan

df["Experience"] = df["minimum_experience"].apply(exp_to_num)
df["Experience"] = df["Experience"].fillna(df["Experience"].mean())
df = df.drop(columns=["minimum_experience"])

In [13]:
def exp_level(x):
    if x <= 1:
        return "Entry"
    elif x <= 3:
        return "Junior"
    elif x <= 5:
        return "Mid"
    else:
        return "Senior"

df["experience_level"] = df["Experience"].apply(exp_level)


##  Cleaning Skills column 

In [14]:
# Step 1: Clean skills
def clean_skills(skill_text):
    # If null
    if pd.isna(skill_text):
        return []
    
    # If it's already a list
    if isinstance(skill_text, list):
        skills = skill_text
    else:
        # Convert to string
        skill_text = str(skill_text).strip()
        if skill_text.lower() == "not specified" or skill_text == "":
            return []
        skills = skill_text.split(",")
    
    # Normalize: lowercase, strip spaces, remove empty strings
    skills = [s.strip().lower() for s in skills if s.strip() != ""]
    return skills

In [15]:
df.head()

,title,functional_area,career_level,minimum_education,skills,Salary,Job Type,City,Experience,experience_level
0,Order Fulfillment Manager,Retail,Experienced Professional,Intermediate/A-Level,"Team Building, Order Tracking, Order Processin...",45000.000000,Full Time,Lahore,2.0,Junior
1,Enterprise Business Development Manager,Other,Not Specified,NaN,"Relationship Building, Sales Acumen, Market Re...",82639.597315,Full Time,Lahore,3.0,Junior
2,Video Editor,Other,Not Specified,NaN,Video Editor,30000.000000,Full Time,Karachi,1.0,Entry
3,Branch in Charge (Patokki),Other,Not Specified,NaN,"Customer Service, Problem Solving, Communicati...",60000.000000,Full Time,Other,3.0,Junior
4,Senior PHP Developer,Other,Not Specified,NaN,"PHP, MySQL, Javascript, Laravel, HTML, CSS, Gi...",82639.597315,Full Time,Lahore,3.0,Junior


# Making A new Dataset Having Clean Required column

In [15]:
df_clean = df[[
    "title",
    "City",
    "Salary",
    "Experience","experience_level",
    "Job Type","functional_area","career_level","skills"
]]
df_clean.head()

,title,City,Salary,Experience,experience_level,Job Type,functional_area,career_level,skills
0,Order Fulfillment Manager,Lahore,45000.000000,2.0,Junior,Full Time,Retail,Experienced Professional,"Team Building, Order Tracking, Order Processin..."
1,Enterprise Business Development Manager,Lahore,82639.597315,3.0,Junior,Full Time,Other,Not Specified,"Relationship Building, Sales Acumen, Market Re..."
2,Video Editor,Karachi,30000.000000,1.0,Entry,Full Time,Other,Not Specified,Video Editor
3,Branch in Charge (Patokki),Other,60000.000000,3.0,Junior,Full Time,Other,Not Specified,"Customer Service, Problem Solving, Communicati..."
4,Senior PHP Developer,Lahore,82639.597315,3.0,Junior,Full Time,Other,Not Specified,"PHP, MySQL, Javascript, Laravel, HTML, CSS, Gi..."


In [23]:
# Save df_clean to CSV
df_clean.to_csv("RozeePk-2024.csv", index=False)

'ls' is not recognized as an internal or external command,
operable program or batch file.
